In [1]:
import datetime

import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader

import pandas as pd
from itertools import product
from IPython.display import clear_output

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
data_dir = 'drive/My Drive/'
train_lang = 'en'

In [5]:
class DatasetSeq(Dataset):
    def __init__(self, data_dir, train_lang='en'):
	#open file
        with open(data_dir + train_lang + '.train', 'r') as f:
            train = f.read().split('\n\n')

        # delete extra tag markup
        train = [x for x in train if not '_ ' in x]
	    #init vocabs of tokens for encoding {<str> token: <int> id}
        self.target_vocab = {} # {p: 1, a: 2, r: 3, pu: 4}
        self.word_vocab = {} # {cat: 1, sat: 2, on: 3, mat: 4, '.': 5}
        self.char_vocab = {} # {c: 1, a: 2, t: 3, ' ': 4, s: 5}
	    
        # Cat sat on mat. -> [1, 2, 3, 4, 5]
        # p    a  r  p pu -> [1, 2, 3, 1, 4]
        # chars  -> [1, 2, 3, 4, 5, 2, 3, 4]

	    #init encoded sequences lists (processed data)
        self.encoded_sequences = []
        self.encoded_targets = []
        self.encoded_char_sequences = []
        # n=1 because first value is padding
        n_word = 1
        n_target = 1
        n_char = 1
        for line in train:
            sequence = []
            target = []
            chars = []
            for item in line.split('\n'):
                if item != '':
                    word, label = item.split(' ')

                    if self.word_vocab.get(word) is None:
                        self.word_vocab[word] = n_word
                        n_word += 1
                    if self.target_vocab.get(label) is None:
                        self.target_vocab[label] = n_target
                        n_target += 1
                    for char in word:
                        if self.char_vocab.get(char) is None:
                            self.char_vocab[char] = n_char
                            n_char += 1
                    sequence.append(self.word_vocab[word])
                    target.append(self.target_vocab[label])
                    chars.append([self.char_vocab[char] for char in word])
            self.encoded_sequences.append(sequence)
            self.encoded_targets.append(target)
            self.encoded_char_sequences.append(chars)

    def __len__(self):
        return len(self.encoded_sequences)

    def __getitem__(self, index):
        return {
            'data': self.encoded_sequences[index], # [1, 2, 3, 4, 6] len=5
            'char': self.encoded_char_sequences[index],# [[1,2,3], [4,5], [1,2], [2,6,5,4], []] len=5
            'target': self.encoded_targets[index], #  (1)
        }

In [6]:
dataset = DatasetSeq(data_dir)

In [7]:
#padding
# seq1 = [1, 2, 3, 4]
# seq2 = [9, 7, 6, 4, 3, 7, 5]
# pad seq1 equal seq2
# seq1 = [1, 2, 3, 4, 0, 0, 0]
# concat(seq1, seq2) [[1, 2, 3, 4, 0, 0, 0],
#                     [9, 7, 6, 4, 3, 7, 5]]

In [8]:
def collate_fn(input_data):
    data = []
    chars = []
    targets = []
    max_len = 0
    for item in input_data:
        if len(item['data']) > max_len:
            max_len = len(item['data'])
        data.append(torch.as_tensor(item['data']))
        chars.append(item['char'])
        targets.append(torch.as_tensor(item['target']))
    chars_seq = [[torch.as_tensor([0]) for _ in range(len(input_data))] for _ in range(max_len)]
    for j in range(len(input_data)):
        for i in range(max_len):
            if len(chars[j]) > i:
                chars_seq[i][j] = torch.as_tensor(chars[j][i])
    for j in range(max_len):
        chars_seq[j] = pad_sequence(chars_seq[j], batch_first=True, padding_value=0)
    data = pad_sequence(data, batch_first=True, padding_value=0)
    targets = pad_sequence(targets, batch_first=True, padding_value=0)
    return {'data': data, 'chars': chars_seq, 'target': targets}

In [9]:
class CharRNN(nn.Module):
    def __init__(self, vocab_size, emb_dim, hidden_dim):
        super().__init__()
        self.char_emb = nn.Embedding(vocab_size, emb_dim)
        self.rnn = nn.GRU(emb_dim, hidden_dim, batch_first=True)

    def forward(self, x):
        emb = self.char_emb(x) # B x T x Emb_dim
        _, out = self.rnn(emb)
        # _: B x T x Hidden 
        # out: 1 x B x Hidden

        return out.transpose(0, 1) # B x 1 x Hidden

# #TODO try to use other RNN archicetures, f.e. RNN and LSTM

In [10]:
class RNNPredictor(nn.Module):
    def __init__(self, vocab_size, emb_dim, hidden_dim, n_classes,
                 char_vocab, char_emb, char_hidden):
        super().__init__()
        #TODO try to use other RNN archicetures, f.e. RNN and LSTM
        self.word_emb = nn.Embedding(vocab_size, emb_dim)
        # batch_first = False: T x B x Vec
        # batch_first = True: B x T x Vec
        self.rnn = nn.RNN(emb_dim + char_hidden, hidden_dim, batch_first=True) 
        self.clf = nn.Linear(hidden_dim, n_classes)
        self.do = nn.Dropout(0.1)
        self.hidden_dim = hidden_dim
        self.char_rnn = CharRNN(char_vocab, char_emb, char_hidden)

    def forward(self, x, chars):
        emb = self.word_emb(x)
        char_features = [self.char_rnn(c.to(x.device)) for c in chars]
        char_features = torch.cat(char_features, dim=1) # конкатенация по времени B x T x Char_hid
        emb = torch.cat((emb, char_features), dim=-1) # конкатенация векторов
        hidden, _ = self.rnn(emb)

        return self.clf(self.do(hidden))


class GRUPredictor(RNNPredictor):
    def __init__(self, vocab_size, emb_dim, hidden_dim, n_classes,
                 char_vocab, char_emb, char_hidden):
        super().__init__(vocab_size, emb_dim, hidden_dim, n_classes,
                 char_vocab, char_emb, char_hidden)
        self.rnn = nn.GRU(emb_dim + char_hidden, hidden_dim, batch_first=True) 

class LSTMPredictor(RNNPredictor):
    def __init__(self, vocab_size, emb_dim, hidden_dim, n_classes,
                 char_vocab, char_emb, char_hidden):
        super().__init__(vocab_size, emb_dim, hidden_dim, n_classes,
                 char_vocab, char_emb, char_hidden)
        self.rnn = nn.LSTM(emb_dim + char_hidden, hidden_dim, batch_first=True)         


In [11]:
#hyper params
vocab_size = len(dataset.word_vocab) + 1
n_classes = len(dataset.target_vocab) + 1
n_chars = len(dataset.char_vocab) + 1
#TODO try to use other model parameters
emb_dim = 256
hidden = 256
char_hid = 64
char_emb = 32
n_epochs = 10
batch_size = 64
cuda_device = 0
batch_size = 100
device = f'cuda:{cuda_device}' if cuda_device != -1 else 'cpu'

In [12]:
models = []
models.append(
     {'name':'GRU-model',
      'model':GRUPredictor(vocab_size, emb_dim, hidden, n_classes, n_chars, char_emb, char_hid).to(device),
      'loss_func':nn.CrossEntropyLoss()
            }
)

models.append(
     {'name':'RNN-model',
      'model':RNNPredictor(vocab_size, emb_dim, hidden, n_classes, n_chars, char_emb, char_hid).to(device),
      'loss_func':nn.CrossEntropyLoss()
            }
)

models.append(
     {'name':'LSTM-model',
      'model':LSTMPredictor(vocab_size, emb_dim, hidden, n_classes, n_chars, char_emb, char_hid).to(device),
      'loss_func':nn.CrossEntropyLoss()
            }
)


for mdl in models:
  mdl['optim'] = torch.optim.Adam(mdl['model'].parameters(), lr=0.001)
  mdl['model'].train()
  print (mdl['name'], mdl['model'])

GRU-model GRUPredictor(
  (word_emb): Embedding(29588, 256)
  (rnn): GRU(320, 256, batch_first=True)
  (clf): Linear(in_features=256, out_features=18, bias=True)
  (do): Dropout(p=0.1, inplace=False)
  (char_rnn): CharRNN(
    (char_emb): Embedding(168, 32)
    (rnn): GRU(32, 64, batch_first=True)
  )
)
RNN-model RNNPredictor(
  (word_emb): Embedding(29588, 256)
  (rnn): RNN(320, 256, batch_first=True)
  (clf): Linear(in_features=256, out_features=18, bias=True)
  (do): Dropout(p=0.1, inplace=False)
  (char_rnn): CharRNN(
    (char_emb): Embedding(168, 32)
    (rnn): GRU(32, 64, batch_first=True)
  )
)
LSTM-model LSTMPredictor(
  (word_emb): Embedding(29588, 256)
  (rnn): LSTM(320, 256, batch_first=True)
  (clf): Linear(in_features=256, out_features=18, bias=True)
  (do): Dropout(p=0.1, inplace=False)
  (char_rnn): CharRNN(
    (char_emb): Embedding(168, 32)
    (rnn): GRU(32, 64, batch_first=True)
  )
)


In [13]:
p_chekpoint = 100

n_steps=len(dataset)//batch_size

df_train_step = pd.DataFrame(columns=['epoch', 'step'], data=product(range(n_epochs), range(0, n_steps, p_chekpoint)))
#df_train_step.set_index(['epoch'], inplace=True)
df_train_step.set_index(['epoch', 'step'], inplace=True)
df_train_step

for mdl in models:
  model = mdl['model']
  optim = mdl['optim']
  loss_func = mdl['loss_func']

  mdl_name = mdl['name']
  #f_train_step[mdl_name+'_loss']=[None]*
  df_train_step.insert(df_train_step.shape[1], mdl_name+'_loss', None)
  df_train_step.insert(df_train_step.shape[1], mdl_name+'_time', None)

  start = datetime.datetime.now()
  for epoch in range(n_epochs):
      dataloader = DataLoader(dataset, 
                              batch_size, 
                              shuffle=True, 
                              collate_fn=collate_fn,
                              drop_last = True,
                              )
      for i, batch in enumerate(dataloader):
          optim.zero_grad()

          predict = model(batch['data'].to(device), batch['chars'])
          loss = loss_func(predict.view(-1, n_classes),
                          batch['target'].to(device).view(-1), 
                          )
          loss.backward()
          optim.step()
          if i % p_chekpoint == 0:
              clear_output(wait=True)
              df_train_step.loc[(epoch, i)][mdl_name+'_loss'] = loss.item()
              df_train_step.loc[(epoch, i)][mdl_name+'_time'] = datetime.datetime.now()-start
              display(df_train_step)
    
      torch.save(model.state_dict(), f'./rnn_chkpt__{mdl_name}_{epoch}.pth')

GRU-model_loss  GRU-model_time RNN-model_loss  RNN-model_time  \
epoch step                                                                 
0     0          3.193892  0:00:01.202440       3.127689  0:00:00.091872   
      100        0.248736  0:00:19.517199       0.158152  0:00:12.805543   
      200        0.190125  0:00:32.142616       0.185377  0:00:25.309484   
1     0          0.122578  0:00:33.754095       0.153994  0:00:26.724815   
      100        0.091903  0:00:46.595711       0.128943  0:00:38.975224   
      200         0.10671  0:01:00.723328       0.109267  0:00:53.391768   
2     0          0.088397  0:01:02.347290       0.041755  0:00:55.089980   
      100        0.064719  0:01:15.847579       0.084755  0:01:07.517807   
      200        0.082728  0:01:28.424689       0.047379  0:01:20.106898   
3     0          0.062054  0:01:29.945284       0.059451  0:01:21.691853   
      100        0.054131  0:01:44.203998       0.063805  0:01:35.687950   
      200        0.071773  0:01:57.196177       0.067235  0:01:48.262881   
4     0            0.0511  0:01:58.569536       0.074258  0:01:49.655858   
      100        0.058759  0:02:11.435055       0.051654  0:02:02.119405   
      200        0.069285  0:02:24.277608       0.050335  0:02:16.200017   
5     0          0.061324  0:02:25.644427       0.063536  0:02:17.990695   
      100        0.033296  0:02:40.475004       0.061333  0:02:31.013101   
      200        0.035063  0:02:52.779784       0.076034  0:02:43.555199   
6     0          0.038611  0:02:54.143510       0.053351  0:02:45.011446   
      100        0.037281  0:03:08.596544       0.061699  0:02:57.879643   
      200        0.053446  0:03:25.105717       0.041633  0:03:11.608217   
7     0          0.041332  0:03:26.388690       0.037445  0:03:13.115840   
      100        0.038056  0:03:39.906642       0.056537  0:03:25.195639   
      200        0.037455  0:03:52.573424       0.042513  0:03:38.535326   
8     0          0.031836  0:03:53.871764        0.02181  0:03:40.067492   
      100        0.029809  0:04:08.727830       0.038779  0:03:53.999451   
      200        0.018203  0:04:21.416618        0.04496  0:04:05.995527   
9     0          0.017984  0:04:22.868222       0.031284  0:04:07.515974   
      100        0.033703  0:04:35.988617       0.026853  0:04:20.502510   
      200         0.02351  0:04:50.208178       0.040158  0:04:32.586992   

           LSTM-model_loss LSTM-model_time  
epoch step                                  
0     0            2.90793  0:00:00.137730  
      100         0.253652  0:00:14.084355  
      200          0.22073  0:00:26.744361  
1     0           0.184448  0:00:28.218634  
      100         0.102279  0:00:41.217333  
      200         0.070369  0:00:55.012668  
2     0           0.078411  0:00:56.484365  
      100         0.099119  0:01:09.413309  
      200         0.074203  0:01:21.961460  
3     0            0.06743  0:01:23.289928  
      100         0.074707  0:01:37.611955  
      200         0.053268  0:01:50.001725  
4     0            0.03707  0:01:51.656421  
      100          0.05187  0:02:04.006410  
      200         0.054733  0:02:17.002793  
5     0           0.065647  0:02:18.343903  
      100         0.053151  0:02:32.613281  
      200         0.056906  0:02:45.358044  
6     0           0.039984  0:02:46.624394  
      100          0.03385  0:02:59.149974  
      200         0.038945  0:03:13.501979  
7     0           0.031682  0:03:14.884433  
      100         0.043492  0:03:27.744983  
      200         0.034509  0:03:40.281313  
8     0           0.032155  0:03:41.924805  
      100         0.029508  0:03:54.447094  
      200         0.045692  0:04:08.871262  
9     0           0.030233  0:04:10.404967  
      100         0.030253  0:04:23.048942  
      200         0.015155  0:04:35.533323

In [18]:
#example

phrase = 'What can you tell me about neural networks ?'
words = phrase.split(' ') 
tokens = [dataset.word_vocab[w] for w in words]
chars = [torch.tensor([dataset.char_vocab[c] for c in w]).unsqueeze(0).to(device) for w in words]

start = datetime.datetime.now()
for mdl in models:
  model = mdl['model']
  print("predict with ", mdl['name'])
  start = datetime.datetime.now()
  with torch.no_grad():
      model.eval()
      predict = model(torch.tensor(tokens).unsqueeze(0).to(device), chars) # 1 x T x N_classes
      labels = torch.argmax(predict, dim=-1).squeeze().cpu().detach().tolist()
      end = datetime.datetime.now() - start

  target_labels = list(dataset.target_vocab.keys())
  print([target_labels[l-1] for l in labels])
  print("Predicted in", datetime.datetime.now()-start, '\n')

predict with  GRU-model
['PRON', 'AUX', 'PRON', 'VERB', 'PRON', 'ADP', 'ADJ', 'NOUN', 'PUNCT']
Predicted in 0:00:00.006137 

predict with  RNN-model
['PRON', 'AUX', 'PRON', 'VERB', 'PRON', 'ADP', 'ADJ', 'NOUN', 'PUNCT']
Predicted in 0:00:00.002750 

predict with  LSTM-model
['PRON', 'AUX', 'PRON', 'VERB', 'PRON', 'ADP', 'ADJ', 'NOUN', 'PUNCT']
Predicted in 0:00:00.003884 



# #TODO try to use other model parameters


In [96]:
def create_models_with_params(p_params):
  emb_dim = p_params['emb_dim']
  hidden = p_params['hidden']
  char_emb = p_params['char_emb']
  char_hid = p_params['char_hid']

  models = []

  models.append(
      {'name':'GRU-model',
        'model':GRUPredictor(vocab_size, emb_dim, hidden, n_classes, n_chars, char_emb, char_hid).to(device),
        'loss_func':nn.CrossEntropyLoss()
              }
  )

  models.append(
      {'name':'RNN-model',
        'model':RNNPredictor(vocab_size, emb_dim, hidden, n_classes, n_chars, char_emb, char_hid).to(device),
        'loss_func':nn.CrossEntropyLoss()
              }
  )

  models.append(
      {'name':'LSTM-model',
        'model':LSTMPredictor(vocab_size, emb_dim, hidden, n_classes, n_chars, char_emb, char_hid).to(device),
        'loss_func':nn.CrossEntropyLoss()
              }
  )


  for mdl in models:
    mdl['optim'] = torch.optim.Adam(mdl['model'].parameters(), lr=0.001)
    mdl['model'].train()

  return (models)


In [127]:
import time

def train_with_params(p_params):

  n_epochs = p_params['n_epochs']
  batch_size = p_params['batch_size']

  p_chekpoint = 100
  n_steps = len(dataset)//batch_size
  p_checked_steps = list(range(0, n_steps, p_chekpoint))
  p_checked_steps.append(n_steps-1)

  models = create_models_with_params(p_params)
  #print (p_checked_steps)
  #time.sleep(10)

  df_train_step = pd.DataFrame(columns=['epoch', 'step'], data=product(range(n_epochs), p_checked_steps))
  #df_train_step.set_index(['epoch'], inplace=True)
  df_train_step.set_index(['epoch', 'step'], inplace=True)

  for mdl in models:
    model = mdl['model']
    optim = mdl['optim']
    loss_func = mdl['loss_func']

    mdl_name = mdl['name']
    df_train_step.insert(df_train_step.shape[1], mdl_name+'_loss', None)
    df_train_step.insert(df_train_step.shape[1], mdl_name+'_time', None)

    start = datetime.datetime.now()
    for epoch in range(n_epochs):
        dataloader = DataLoader(dataset, 
                                batch_size, 
                                shuffle=True, 
                                collate_fn=collate_fn,
                                drop_last = True,
                                )
        for i, batch in enumerate(dataloader):
            optim.zero_grad()

            predict = model(batch['data'].to(device), batch['chars'])
            loss = loss_func(predict.view(-1, n_classes),
                            batch['target'].to(device).view(-1), 
                            )
            loss.backward()
            optim.step()
            if (i % p_chekpoint == 0) or (i==n_steps-1):
                clear_output(wait=True)
                df_train_step.loc[(epoch, i)][mdl_name+'_loss'] = loss.item()
                df_train_step.loc[(epoch, i)][mdl_name+'_time'] = datetime.datetime.now()-start
                display(df_train_step)
        
            torch.save(model.state_dict(), f'./rnn_chkpt__{mdl_name}_{epoch}.pth')
    
  return(models, df_train_step)

In [128]:
params1 = {
    'emb_dim':128,
    'hidden':128,
    'char_hid':32,
    'char_emb':16,
    'n_epochs':5,
    'batch_size':50
         }
models1, df1 = train_with_params(params1)


GRU-model_loss  GRU-model_time RNN-model_loss  RNN-model_time  \
epoch step                                                                 
0     0          3.039315  0:00:00.095831        2.72506  0:00:00.066779   
      100        0.544996  0:00:11.988813       0.430627  0:00:11.478546   
      200        0.350741  0:00:23.743414       0.251834  0:00:22.951847   
      300        0.271577  0:00:35.384090       0.203385  0:00:34.222999   
      400        0.212486  0:00:47.489843       0.188354  0:00:46.973405   
      423        0.171084  0:00:50.042464       0.214683  0:00:49.573828   
1     0          0.168799  0:00:50.163937       0.154384  0:00:49.721697   
      100        0.227883  0:01:03.176063       0.232015  0:01:01.308725   
      200        0.159651  0:01:14.682748       0.168696  0:01:12.266275   
      300         0.19347  0:01:26.263878        0.16792  0:01:22.958835   
      400        0.107388  0:01:37.014958       0.117946  0:01:34.167116   
      423        0.100655  0:01:40.192548       0.125854  0:01:37.001693   
2     0          0.114002  0:01:40.360188       0.177203  0:01:37.166521   
      100        0.085701  0:01:51.666581       0.167636  0:01:49.626278   
      200        0.139407  0:02:04.898496       0.088082  0:02:01.111259   
      300         0.11794  0:02:16.386372       0.072217  0:02:12.293638   
      400        0.143042  0:02:28.022475       0.127011  0:02:23.987566   
      423        0.071743  0:02:30.305378       0.157197  0:02:26.577368   
3     0          0.078554  0:02:30.436746        0.05068  0:02:26.745139   
      100        0.074881  0:02:42.394819       0.066196  0:02:38.206241   
      200        0.060404  0:02:54.634084       0.102537  0:02:51.619509   
      300        0.067416  0:03:07.058292       0.049855  0:03:02.937323   
      400        0.073449  0:03:18.088155       0.109431  0:03:13.157714   
      423        0.061812  0:03:20.683213       0.071327  0:03:16.716456   
4     0          0.091006  0:03:20.783083        0.07704  0:03:16.825512   
      100        0.057305  0:03:32.699355       0.073108  0:03:27.218961   
      200        0.054006  0:03:43.155064       0.085854  0:03:38.531642   
      300        0.094487  0:03:55.902411       0.067758  0:03:51.324880   
      400        0.091802  0:04:07.811028       0.050499  0:04:03.337721   
      423        0.058968  0:04:10.221874       0.054406  0:04:05.703591   

           LSTM-model_loss LSTM-model_time  
epoch step                                  
0     0           3.028796  0:00:00.071287  
      100         0.483895  0:00:11.582726  
      200         0.287818  0:00:23.353007  
      300         0.220455  0:00:35.433829  
      400         0.180906  0:00:48.482774  
      423          0.17792  0:00:51.146041  
1     0           0.210726  0:00:51.273873  
      100         0.083444  0:01:02.833395  
      200         0.134459  0:01:14.566067  
      300         0.125301  0:01:25.519030  
      400         0.149515  0:01:36.791853  
      423         0.087338  0:01:40.704767  
2     0           0.077358  0:01:40.984227  
      100         0.073118  0:01:53.130388  
      200         0.120494  0:02:05.407121  
      300         0.117534  0:02:17.015988  
      400         0.065489  0:02:28.401460  
      423         0.092269  0:02:31.166504  
3     0           0.130452  0:02:31.283914  
      100         0.080537  0:02:44.453005  
      200         0.080076  0:02:56.304450  
      300         0.095153  0:03:08.045525  
      400         0.077774  0:03:18.717487  
      423         0.067588  0:03:21.954088  
4     0            0.07845  0:03:22.061039  
      100         0.091902  0:03:33.381138  
      200          0.05264  0:03:46.802971  
      300         0.072991  0:03:57.648409  
      400         0.067766  0:04:09.460848  
      423          0.08687  0:04:11.716039

In [129]:
params2 = {
    'emb_dim':128,
    'hidden':128,
    'char_hid':32,
    'char_emb':16,
    'n_epochs':5,
    'batch_size':100
         }
models2, df2 = train_with_params(params2)

GRU-model_loss  GRU-model_time RNN-model_loss  RNN-model_time  \
epoch step                                                                 
0     0           3.03403  0:00:00.139864       3.211715  0:00:00.191399   
      100         0.42468  0:00:17.923698       0.427071  0:00:17.258502   
      200        0.255159  0:00:33.699505       0.274083  0:00:34.617552   
      211        0.239572  0:00:35.232081       0.238593  0:00:36.202610   
1     0          0.116543  0:00:35.649792       0.216045  0:00:36.376841   
      100        0.165558  0:00:51.371329       0.223507  0:00:52.341978   
      200        0.119555  0:01:08.258113       0.140223  0:01:08.308561   
      211        0.133414  0:01:10.823971       0.152728  0:01:10.791060   
2     0          0.195927  0:01:10.993641       0.118567  0:01:11.077296   
      100        0.115097  0:01:28.072689       0.145745  0:01:28.287371   
      200        0.123749  0:01:43.493787       0.152376  0:01:43.969352   
      211        0.134202  0:01:45.000066       0.141666  0:01:45.567535   
3     0          0.101159  0:01:45.148729       0.141321  0:01:45.698348   
      100        0.062546  0:02:01.424730       0.055888  0:02:01.412536   
      200        0.114459  0:02:19.766439       0.063155  0:02:17.525580   
      211        0.098027  0:02:21.249419       0.059206  0:02:21.270006   
4     0          0.062687  0:02:21.429672       0.056552  0:02:21.642744   
      100         0.07699  0:02:37.503990       0.081061  0:02:38.159661   
      200        0.061952  0:02:53.600302       0.097384  0:02:53.674270   
      211        0.070728  0:02:55.348102       0.073214  0:02:55.441591   

           LSTM-model_loss LSTM-model_time  
epoch step                                  
0     0           2.940763  0:00:00.095111  
      100         0.293812  0:00:15.881462  
      200         0.273257  0:00:33.812059  
      211         0.266679  0:00:35.960275  
1     0           0.149972  0:00:36.172681  
      100         0.191555  0:00:52.486704  
      200         0.173397  0:01:08.271320  
      211         0.111108  0:01:09.836582  
2     0           0.123361  0:01:10.002208  
      100         0.139531  0:01:27.622541  
      200         0.105848  0:01:44.485427  
      211         0.119318  0:01:46.235363  
3     0           0.107648  0:01:46.381568  
      100         0.107413  0:02:02.573802  
      200         0.085672  0:02:18.895572  
      211          0.10667  0:02:20.696960  
4     0           0.071789  0:02:20.873308  
      100         0.075131  0:02:39.428751  
      200         0.092632  0:02:55.504104  
      211         0.066475  0:02:57.068695

In [130]:
params3 = {
    'emb_dim':16,
    'hidden':16,
    'char_hid':8,
    'char_emb':8,
    'n_epochs':5,
    'batch_size':100
         }
models3, df3 = train_with_params(params3)


GRU-model_loss  GRU-model_time RNN-model_loss  RNN-model_time  \
epoch step                                                                 
0     0          2.829763  0:00:00.434135       3.246931  0:00:00.324276   
      100         1.18076  0:00:12.805156       1.173164  0:00:12.681141   
      200        0.415626  0:00:26.352919       0.895554  0:00:25.208407   
      211        0.867672  0:00:27.524575       0.994369  0:00:26.291256   
1     0           0.91011  0:00:27.623233       0.786917  0:00:26.405146   
      100        0.583334  0:00:40.351299       0.923742  0:00:38.603546   
      200         0.66404  0:00:52.772741       0.440387  0:00:50.850335   
      211        0.388813  0:00:54.095199       0.758145  0:00:52.348477   
2     0          0.504345  0:00:54.249370       0.787524  0:00:52.446596   
      100        0.663982  0:01:06.552324       0.551559  0:01:05.831569   
      200        0.453712  0:01:20.643694       0.477269  0:01:19.041949   
      211        0.473658  0:01:21.826975       0.457401  0:01:20.516100   
3     0          0.400519  0:01:21.960164       0.389843  0:01:20.654714   
      100        0.246278  0:01:34.004017         0.4545  0:01:33.197543   
      200        0.360806  0:01:46.134032        0.35367  0:01:46.325654   
      211        0.228902  0:01:47.471756       0.431352  0:01:47.477102   
4     0          0.381435  0:01:47.754940       0.485687  0:01:47.580574   
      100        0.339709  0:01:59.654955       0.347995  0:02:01.435679   
      200        0.274739  0:02:12.732566       0.388899  0:02:13.787787   
      211        0.268323  0:02:14.597305       0.323195  0:02:15.045859   

           LSTM-model_loss LSTM-model_time  
epoch step                                  
0     0           2.931488  0:00:00.102295  
      100         0.794211  0:00:12.497990  
      200         0.862722  0:00:24.737916  
      211         0.566258  0:00:25.892344  
1     0           0.815184  0:00:26.007347  
      100         0.826715  0:00:38.076363  
      200         0.500231  0:00:51.604054  
      211         0.769255  0:00:52.643900  
2     0           0.493501  0:00:52.771275  
      100         0.350689  0:01:05.066402  
      200         0.354098  0:01:17.067470  
      211         0.465051  0:01:18.466825  
3     0           0.158858  0:01:18.815898  
      100          0.31776  0:01:31.208753  
      200          0.35059  0:01:43.707457  
      211         0.400954  0:01:46.033374  
4     0           0.483016  0:01:46.194290  
      100         0.331573  0:01:58.405928  
      200         0.273906  0:02:09.796587  
      211         0.390976  0:02:11.059286

In [135]:
params4 = {
    'emb_dim':320,
    'hidden':320,
    'char_hid':128,
    'char_emb':64,
    'n_epochs':5,
    'batch_size':100
         }
models4, df4 = train_with_params(params4)


GRU-model_loss  GRU-model_time RNN-model_loss  RNN-model_time  \
epoch step                                                                 
0     0          2.694156  0:00:00.146078       3.212856  0:00:00.329527   
      100         0.24067  0:00:28.594018       0.191581  0:00:30.103578   
      200        0.124785  0:00:58.009449       0.120272  0:00:57.350324   
      211        0.126291  0:01:00.894159       0.152567  0:01:00.411495   
1     0          0.115327  0:01:01.125444       0.106734  0:01:00.717054   
      100        0.059679  0:01:28.669126       0.104084  0:01:29.382278   
      200        0.054877  0:01:58.753680       0.090072  0:01:57.486607   
      211        0.056294  0:02:01.635955       0.116443  0:02:00.280437   
2     0          0.065782  0:02:02.054375       0.106052  0:02:00.501620   
      100         0.08541  0:02:30.987384       0.065234  0:02:30.200902   
      200         0.07016  0:03:00.823604         0.0784  0:02:57.292698   
      211        0.045455  0:03:04.296702       0.088535  0:03:00.039631   
3     0          0.049721  0:03:04.579324       0.053812  0:03:00.322598   
      100        0.050146  0:03:32.939581       0.072221  0:03:29.620069   
      200        0.060296  0:04:01.652274       0.060964  0:03:57.064415   
      211        0.048352  0:04:04.757408       0.072287  0:03:59.975086   
4     0          0.056461  0:04:04.989126       0.049792  0:04:00.322674   
      100          0.0383  0:04:34.131231       0.074064  0:04:29.355715   
      200        0.050603  0:05:02.438649       0.052652  0:04:56.990862   
      211        0.042402  0:05:06.036539       0.055118  0:05:00.155144   

           LSTM-model_loss LSTM-model_time  
epoch step                                  
0     0           2.867287  0:00:00.117051  
      100         0.182169  0:00:29.408637  
      200         0.173129  0:00:57.027461  
      211         0.097462  0:00:59.967492  
1     0           0.118612  0:01:00.219005  
      100         0.087427  0:01:30.065716  
      200          0.07083  0:01:58.227267  
      211         0.079915  0:02:01.149746  
2     0           0.072639  0:02:01.403275  
      100         0.072509  0:02:31.225238  
      200         0.063758  0:02:59.244556  
      211         0.067687  0:03:03.165022  
3     0           0.073355  0:03:03.417806  
      100         0.069048  0:03:33.016010  
      200         0.063885  0:04:01.135698  
      211         0.048669  0:04:03.853476  
4     0           0.069456  0:04:04.090707  
      100         0.049992  0:04:32.578019  
      200         0.048386  0:05:00.927155  
      211         0.049522  0:05:03.863897

In [143]:
params5 = {
    'emb_dim':8,
    'hidden':16,
    'char_hid':32,
    'char_emb':32,
    'n_epochs':5,
    'batch_size':100
         }
models5, df5 = train_with_params(params5)

GRU-model_loss  GRU-model_time RNN-model_loss  RNN-model_time  \
epoch step                                                                 
0     0          3.083146  0:00:00.075556       3.372885  0:00:00.096985   
      100        0.885262  0:00:12.515627       1.134809  0:00:12.739047   
      200        0.656762  0:00:28.148276       0.835395  0:00:24.872564   
      211        0.817306  0:00:29.400814       0.595941  0:00:26.426501   
1     0          0.667592  0:00:29.572407       0.562732  0:00:26.584309   
      100        0.746458  0:00:42.523818       0.663458  0:00:39.491428   
      200        0.670283  0:00:56.717466       0.385001  0:00:51.675866   
      211        0.518611  0:00:57.891117       0.720851  0:00:53.198189   
2     0          0.498963  0:00:58.056453       0.432647  0:00:53.372261   
      100        0.690586  0:01:10.701641       0.676492  0:01:06.930420   
      200         0.41551  0:01:24.702301        0.45036  0:01:19.364832   
      211        0.450537  0:01:25.924503       0.436715  0:01:20.695655   
3     0          0.545393  0:01:26.037330       0.357024  0:01:20.836898   
      100        0.505399  0:01:38.611798       0.344317  0:01:33.284278   
      200        0.567836  0:01:50.907912       0.417845  0:01:44.989836   
      211        0.498015  0:01:52.171214       0.313575  0:01:47.005579   
4     0          0.352061  0:01:52.312112       0.279847  0:01:47.152816   
      100        0.474238  0:02:04.860156       0.253617  0:02:00.652697   
      200        0.418696  0:02:18.546449       0.354238  0:02:12.359268   
      211         0.44826  0:02:20.555539       0.153702  0:02:14.220461   

           LSTM-model_loss LSTM-model_time  
epoch step                                  
0     0           3.254342  0:00:00.332056  
      100         0.839311  0:00:12.773745  
      200         0.881179  0:00:24.203094  
      211         0.673644  0:00:26.009876  
1     0           0.560402  0:00:26.485621  
      100         0.760232  0:00:38.474897  
      200         0.586183  0:00:52.165394  
      211         0.802959  0:00:53.631835  
2     0           0.837818  0:00:53.797512  
      100          0.66054  0:01:05.861698  
      200         0.533323  0:01:18.458546  
      211         0.351976  0:01:19.801524  
3     0           0.585677  0:01:19.906474  
      100         0.551626  0:01:32.704262  
      200         0.480013  0:01:46.694914  
      211         0.344117  0:01:47.842866  
4     0           0.452162  0:01:47.966346  
      100         0.316374  0:02:00.567228  
      200         0.291687  0:02:12.864026  
      211         0.473617  0:02:14.087845

In [144]:
params6 = {
    'emb_dim':512,
    'hidden':512,
    'char_hid':128,
    'char_emb':128,
    'n_epochs':5,
    'batch_size':50
         }
models6, df6 = train_with_params(params6)

GRU-model_loss  GRU-model_time RNN-model_loss  RNN-model_time  \
epoch step                                                                 
0     0           2.94926  0:00:00.119967       2.824761  0:00:00.133562   
      100         0.16889  0:00:33.092209       0.195873  0:00:30.972818   
      200          0.1016  0:01:04.900940       0.133723  0:01:01.229593   
      300        0.118869  0:01:36.915442       0.123901  0:01:31.609142   
      400         0.06371  0:02:10.142664       0.113115  0:02:02.106415   
      423        0.089546  0:02:16.889286       0.108538  0:02:09.648738   
1     0          0.087218  0:02:17.219755       0.075693  0:02:09.953770   
      100        0.103156  0:02:48.004566       0.063863  0:02:38.605466   
      200        0.095278  0:03:21.841042       0.089778  0:03:09.314029   
      300        0.061152  0:03:53.013020       0.100895  0:03:38.955711   
      400        0.079076  0:04:25.519210        0.08102  0:04:09.194366   
      423        0.065949  0:04:33.067883       0.108491  0:04:16.366352   
2     0          0.056359  0:04:33.392801       0.056328  0:04:16.641667   
      100        0.084675  0:05:05.453057       0.048094  0:04:45.492536   
      200        0.044798  0:05:37.048960        0.06008  0:05:15.512374   
      300          0.0571  0:06:09.752832        0.04762  0:05:45.469374   
      400        0.062425  0:06:40.724111       0.048804  0:06:16.621051   
      423        0.051186  0:06:47.854267       0.030249  0:06:23.190246   
3     0          0.058411  0:06:48.225684       0.053576  0:06:23.470552   
      100        0.033924  0:07:20.698097       0.057031  0:06:54.113001   
      200        0.043355  0:07:52.851477       0.058832  0:07:23.606639   
      300        0.032492  0:08:24.239547        0.03116  0:07:55.226931   
      400        0.035389  0:08:56.089339        0.07457  0:08:24.874099   
      423        0.037996  0:09:03.614815       0.044954  0:08:31.580831   
4     0          0.019872  0:09:03.920287       0.024266  0:08:31.991549   
      100        0.020547  0:09:37.290298       0.020148  0:09:01.885997   
      200        0.033076  0:10:09.643324       0.050013  0:09:32.180386   
      300        0.018742  0:10:41.674086       0.038653  0:10:02.685389   
      400        0.023291  0:11:12.972179       0.050659  0:10:33.891062   
      423         0.03738  0:11:20.526839       0.048977  0:10:40.443477   

           LSTM-model_loss LSTM-model_time  
epoch step                                  
0     0           2.811168  0:00:00.105512  
      100         0.209588  0:00:32.552054  
      200         0.158092  0:01:05.631413  
      300         0.129413  0:01:38.169803  
      400         0.160888  0:02:10.976412  
      423         0.095988  0:02:18.084801  
1     0            0.12145  0:02:18.389754  
      100         0.098871  0:02:51.688956  
      200         0.104499  0:03:23.767144  
      300         0.077038  0:03:56.253060  
      400         0.063363  0:04:30.150504  
      423         0.085107  0:04:36.983504  
2     0           0.069084  0:04:37.356373  
      100         0.049532  0:05:09.468674  
      200         0.055135  0:05:42.697406  
      300         0.024823  0:06:15.832069  
      400         0.071481  0:06:47.302740  
      423         0.062518  0:06:55.273228  
3     0           0.060557  0:06:55.573380  
      100         0.034232  0:07:28.919534  
      200          0.03682  0:08:01.478383  
      300         0.037116  0:08:34.446449  
      400          0.03663  0:09:06.791942  
      423         0.032613  0:09:14.534240  
4     0           0.020004  0:09:14.850178  
      100         0.032887  0:09:46.376614  
      200         0.034278  0:10:19.913088  
      300         0.024013  0:10:53.227204  
      400         0.032169  0:11:25.694054  
      423         0.029212  0:11:33.414501

In [145]:
print('params1:', params1)
print('params2:', params2)
print('params3:', params3)
print('params4:', params4)
print('params5:', params5)
print('params6:', params6)

params1: {'emb_dim': 128, 'hidden': 128, 'char_hid': 32, 'char_emb': 16, 'n_epochs': 5, 'batch_size': 50}
params2: {'emb_dim': 128, 'hidden': 128, 'char_hid': 32, 'char_emb': 16, 'n_epochs': 5, 'batch_size': 100}
params3: {'emb_dim': 16, 'hidden': 16, 'char_hid': 8, 'char_emb': 8, 'n_epochs': 5, 'batch_size': 100}
params4: {'emb_dim': 320, 'hidden': 320, 'char_hid': 128, 'char_emb': 64, 'n_epochs': 5, 'batch_size': 100}
params5: {'emb_dim': 8, 'hidden': 16, 'char_hid': 32, 'char_emb': 32, 'n_epochs': 5, 'batch_size': 100}
params6: {'emb_dim': 512, 'hidden': 512, 'char_hid': 128, 'char_emb': 128, 'n_epochs': 5, 'batch_size': 50}


In [146]:
df = pd.DataFrame(columns = df1.columns)
df.loc['params1']=df1.iloc[-1]
df.loc['params2']=df2.iloc[-1]
df.loc['params3']=df3.iloc[-1]
df.loc['params4']=df4.iloc[-1]
df.loc['params5']=df5.iloc[-1]
df.loc['params6']=df6.iloc[-1]
df

,GRU-model_loss,GRU-model_time,RNN-model_loss,RNN-model_time,LSTM-model_loss,LSTM-model_time
params1,0.058968,0 days 00:04:10.221874,0.054406,0 days 00:04:05.703591,0.086870,0 days 00:04:11.716039
params2,0.070728,0 days 00:02:55.348102,0.073214,0 days 00:02:55.441591,0.066475,0 days 00:02:57.068695
params3,0.382366,0 days 00:02:13.209600,0.292268,0 days 00:02:14.932619,0.313288,0 days 00:02:15.261289
params4,0.042402,0 days 00:05:06.036539,0.055118,0 days 00:05:00.155144,0.049522,0 days 00:05:03.863897
params5,0.448260,0 days 00:02:20.555539,0.153702,0 days 00:02:14.220461,0.473617,0 days 00:02:14.087845
params6,0.037380,0 days 00:11:20.526839,0.048977,0 days 00:10:40.443477,0.029212,0 days 00:11:33.414501
